In [1]:
import geopandas as gpd
import pandas as pd
import geopandas
import shapely.geometry as geom
from polygon_geohasher.polygon_geohasher import polygon_to_geohashes, geohashes_to_polygon
import pygeohash as gh
%matplotlib inline
import geohash
import geopandas as gp
import pandas as pd
import math
import geojson
from geojson import MultiLineString
from shapely.geometry import MultiLineString
from shapely.geometry import MultiPoint
import numpy as np
from shapely import geometry
from shapely.geometry import Point, Polygon, box,LineString
from geopandas import datasets, GeoDataFrame, read_file
from geopandas.tools import overlay
from matplotlib import pyplot
import matplotlib.pyplot as plt
from shapely.geometry.polygon import LinearRing, Polygon
from numpy import cos,sin,arccos
import numpy as np
from pylab import *
from shapely.ops import cascaded_union
from shapely.geometry import Point, LineString, mapping
import scipy.interpolate
import geopy
import geopy.distance
import time
import networkx as nx
from sklearn.model_selection import train_test_split
import datetime as dt

In [10]:
processed_geohash = pd.read_csv('processed_geohash_6.csv')

In [11]:
processed_geohash.head()

,trj_id,geohash_source,geohash_dest,timestamp_source,timestamp_dest,time_taken,hour,day
0,100,w21zth,w21zt5,2019-04-16 02:44:15,2019-04-16 02:44:53,38.0,2,1
1,100,w21zt5,w21zsg,2019-04-16 02:44:50,2019-04-16 02:48:14,204.0,2,1
2,100,w21zsg,w21zsf,2019-04-16 02:48:06,2019-04-16 02:48:42,36.0,2,1
3,100,w21zsf,w21zsd,2019-04-16 02:48:35,2019-04-16 02:49:07,32.0,2,1
4,100,w21zsd,w21zs9,2019-04-16 02:49:03,2019-04-16 02:49:37,34.0,2,1


In [12]:
processed_geohash['node_trip'] = processed_geohash['geohash_source'] + '-' + processed_geohash['geohash_dest']

In [13]:
len(unique(processed_geohash['node_trip']))

3904

In [14]:
processed_geohash.sort_values(by=['node_trip'])

,trj_id,geohash_source,geohash_dest,timestamp_source,timestamp_dest,time_taken,hour,day,node_trip
188742,42292,w21xjz,w21xjy,2019-04-15 07:06:51,2019-04-15 07:07:35,44.0,7,0,w21xjz-w21xjy
2753,10452,w21xjz,w21xjy,2019-04-21 10:39:55,2019-04-21 10:40:38,43.0,10,6,w21xjz-w21xjy
274410,58735,w21xjz,w21xmb,2019-04-20 14:54:20,2019-04-20 14:55:32,72.0,14,5,w21xjz-w21xmb
188741,42292,w21xmb,w21xjz,2019-04-15 07:05:23,2019-04-15 07:06:59,96.0,7,0,w21xmb-w21xjz
2752,10452,w21xmb,w21xjz,2019-04-21 10:38:41,2019-04-21 10:40:04,83.0,10,6,w21xmb-w21xjz
...,...,...,...,...,...,...,...,...,...
404738,67302,w23bh9,w23bh3,2019-04-19 14:39:53,2019-04-19 14:40:18,25.0,14,4,w23bh9-w23bh3
372998,65026,w23bh9,w23bh3,2019-04-19 03:31:29,2019-04-19 03:32:03,34.0,3,4,w23bh9-w23bh3
372996,65026,w23bh9,w23bhd,2019-04-19 03:26:28,2019-04-19 03:27:06,38.0,3,4,w23bh9-w23bhd
34124,17556,w23bh9,w23bhd,2019-04-20 18:35:59,2019-04-20 18:36:36,37.0,18,5,w23bh9-w23bhd


In [15]:
busy_hours = [8, 9, 10, 17, 18, 19]

In [16]:
avg_mean = pd.DataFrame(processed_geohash.groupby('hour').mean().reset_index())
avg_mean.head(24)

,hour,trj_id,time_taken,day
0,0,50687.796367,60.256002,2.645474
1,1,49638.413786,56.865116,2.759070
2,2,50835.579811,51.709476,3.187310
3,3,48747.385876,49.894776,3.448318
4,4,50359.766997,48.866705,3.544025
5,5,51390.098885,49.141421,3.375976
6,6,51690.051440,51.364607,3.408308
7,7,50519.985637,51.500139,3.231326
8,8,52422.233732,52.361007,3.189928
9,9,50606.519577,57.805919,3.036103


In [17]:
weekend_num = [5,6]
weekend_geohash = processed_geohash[processed_geohash['day'].isin(weekend_num)]
#weekend_visuals = pd.DataFrame(weekend_visuals.groupby('hour').mean().reset_index())
#weekend_visuals.head(24)

In [18]:
weekday_num = [0,1,2,3,4]
weekday_geohash = processed_geohash[processed_geohash['day'].isin(weekday_num)]


In [19]:
weekend_geohash_busy = weekend_geohash[weekend_geohash['hour'].isin(busy_hours)]
weekend_geohash_normal = weekend_geohash[~weekend_geohash['hour'].isin(busy_hours)]
weekday_geohash_busy = weekday_geohash[weekday_geohash['hour'].isin(busy_hours)]
weekday_geohash_normal = weekday_geohash[~weekday_geohash['hour'].isin(busy_hours)]

In [20]:
weekend_geohash_busy.to_csv('weekend_geohash_busy_6.csv', index=False)
weekend_geohash_busy.to_csv('weekend_geohash_normal_6.csv', index=False)
weekday_geohash_busy.to_csv('weekday_geohash_busy_6.csv', index=False)
weekday_geohash_busy.to_csv('weekday_geohash_normal_6.csv', index=False)


In [54]:
mean_hours = pd.concat([avg_mean, weekend_visuals, weekday_visuals], axis=1)
mean_hours.columns = ['hour_mean', 'trj_id_mean', 'time_taken_mean', 'day_mean',
                      'hour_weekend', 'trj_id_weekend', 'time_taken_weekend', 'day_weekend',
                      'hour_weekday', 'trj_id_weekday', 'time_taken_weekday', 'day_weekday']
mean_hours = mean_hours.drop(['trj_id_mean', 'day_mean', 'trj_id_weekend', 'day_weekend', 'trj_id_weekday', 'day_weekday'], axis=1)
mean_hours.head(24)

,hour_mean,time_taken_mean,hour_weekend,time_taken_weekend,hour_weekday,time_taken_weekday
0,0,9.674776,0,7.319870,0,10.348156
1,1,9.070952,1,8.114467,1,9.384057
2,2,8.255209,2,8.071091,2,8.353419
3,3,8.018932,3,8.094605,3,7.968931
4,4,7.826576,4,8.012091,4,7.684021
5,5,7.914475,5,8.120439,5,7.762681
6,6,8.287010,6,8.441822,6,8.176955
7,7,8.279104,7,8.365048,7,8.235841
8,8,8.454514,8,8.284109,8,8.525369
9,9,9.360461,9,8.528386,9,9.584592


In [57]:
weekend_busy_hours = mean_hours[mean_hours['time_taken_weekend'] > mean_hours['time_taken_mean']]
weekend_busy_hours.head(24)

,hour_mean,time_taken_mean,hour_weekend,time_taken_weekend,hour_weekday,time_taken_weekday
3,3,8.018932,3,8.094605,3,7.968931
4,4,7.826576,4,8.012091,4,7.684021
5,5,7.914475,5,8.120439,5,7.762681
6,6,8.287010,6,8.441822,6,8.176955
7,7,8.279104,7,8.365048,7,8.235841
13,13,8.105596,13,8.410920,13,7.976156
14,14,7.774637,14,7.824988,14,7.752943
15,15,7.494910,15,7.624973,15,7.425927
16,16,7.265731,16,7.463924,16,7.162772
17,17,6.969233,17,7.364095,17,6.779105
